In [23]:
from PIL import Image
import requests as rq
from io import BytesIO
import pandas as pd
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

data = pd.read_csv("data_sejal.csv")
images = []
captions = data.caption
for url in data.photo:
    response = rq.get(url)
    img = Image.open(BytesIO(response.content)).resize((224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    images.append(x)

        username                                              photo  \
0  eestevens0131  https://scontent-arn2-1.cdninstagram.com/vp/b2...   
1   brooke.olson  https://scontent-arn2-1.cdninstagram.com/vp/84...   
2   brooke.olson  https://scontent-arn2-1.cdninstagram.com/vp/a7...   
3   brooke.olson  https://scontent-arn2-1.cdninstagram.com/vp/2e...   
4   brooke.olson  https://scontent-arn2-1.cdninstagram.com/vp/a3...   

                                             caption  
0                   Still dysfunctional after take 2  
1                            Heyyyyyyy rose #chicago  
2                      Friendos on the fourth!🏄🏽‍♀️🎉  
3  Congrats Laur!! So proud of everything you hav...  
4                                  Love my friendos💎  
